In [6]:
%cd /mnt/c/Users/recre/OneDrive/CS583

/mnt/c/Users/recre/OneDrive/CS583


In [190]:
import numpy as np
import pandas as pd

In [268]:
s = pd.read_csv("proj1_input-data.txt", header = None, sep = "\t",squeeze = True)
s

0                   {20, 30, 80, 70, 50, 90}
1                           {20, 10, 80, 70}
2                               {10, 20, 80}
3                               {20, 30, 80}
4                                   {20, 80}
5    {20, 30, 80, 70, 50, 90, 100, 120, 140}
Name: 0, dtype: object

In [254]:
da = s.str[1:-1].str.get_dummies(sep = ", ").astype("int8").rename(columns = lambda x: int(x)).sort_index(axis = 1)
da

,10,20,30,50,70,80,90,100,120,140
0,0,1,1,1,1,1,1,0,0,0
1,1,1,0,0,1,1,0,0,0,0
2,1,1,0,0,0,1,0,0,0,0
3,0,1,1,0,0,1,0,0,0,0
4,0,1,0,0,0,1,0,0,0,0
5,0,1,1,1,1,1,1,1,1,1


In [156]:
argl = [i.rstrip("\n") for i in open("proj1_parameter-file.txt", "r")]
argl

['MIS(10) = 0.43',
 'MIS(20) = 0.30',
 'MIS(30) = 0.30',
 'MIS(40) = 0.40',
 'MIS(50) = 0.40',
 'MIS(60) = 0.30',
 'MIS(70) = 0.20',
 'MIS(80) = 0.20',
 'MIS(90) = 0.20',
 'MIS(100) = 0.10',
 'MIS(120) = 0.20',
 'MIS(140) = 0.15',
 'SDC = 0.1',
 'cannot_be_together: {20, 40}, {70, 80}',
 'must-have: 20 or 40 or 50']

In [255]:
ms = pd.Series(dict([(int(j[0][4:-1]), float(j[1])) for j in [i.split(" = ") for i in argl if i.startswith("MIS")]]))
ms

10     0.43
20     0.30
30     0.30
40     0.40
50     0.40
60     0.30
70     0.20
80     0.20
90     0.20
100    0.10
120    0.20
140    0.15
dtype: float64

In [286]:
def mis(L):
    op = np.array([ms[i].min() for i in L])
    return(op)

In [293]:
def support(L):
    op = pd.concat([da.loc[:, i].all(axis = 1) for i in L], axis = 1).mean().values
    return(op)

In [306]:
L = np.array([[i] for i in da.columns[da.mean() > ms.min()]])
L

array([[ 10],
       [ 20],
       [ 30],
       [ 50],
       [ 70],
       [ 80],
       [ 90],
       [100],
       [120],
       [140]])

In [307]:
Lcount = support(L)
Lcount

array([ 0.33333333,  1.        ,  0.5       ,  0.33333333,  0.5       ,
        1.        ,  0.33333333,  0.16666667,  0.16666667,  0.16666667])

In [308]:
mis(L)

array([ 0.43,  0.3 ,  0.3 ,  0.4 ,  0.2 ,  0.2 ,  0.2 ,  0.1 ,  0.2 ,  0.15])

In [319]:
F = [L[Lcount > mis(L)]]
F

[array([[ 20],
        [ 30],
        [ 70],
        [ 80],
        [ 90],
        [100],
        [140]])]

In [321]:
Lsub = L[support(L) > mis(L)]
Lsub

array([[ 20],
       [ 30],
       [ 70],
       [ 80],
       [ 90],
       [100],
       [140]])

In [ ]:
def l2_candidate_gen(L, ms, phi):
    Lsub = L[support(L) > mis(L)]